# Import modules

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

import PIL
import matplotlib.pyplot as plt

import copy

# Set GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Image to Tensor 

In [ ]:
def image_loader(img_path, img_size):
    loader = transforms.Compose([
     transforms.Resize(img_size), # Resize image size
     transforms.ToTensor() # Make Tensor
    ])
    
    img = PIL.Image.open(img_path)
    
    # Add dimension to img to make batch for model_input
    img = loader(img).unsqueeze(0)
    return img.to(device, torch.float) # on GPU

# torch.Tensor -> img to display

In [ ]:
def imshow(tensor):
    # matplotlib is based on cpu
    image = tensor.cpu().clone()
    # Remove dimension for batch 
    image = image.squeeze(0)
    # to PIL
    image = transforms.ToPILImage()(image)
    
    # Display
    plt.imshow(image)
    plt.show()

# Image Reconstruction

* Optimizing image means updating to get lower total loss<br>
* using MSE as Loss_func, make noise image to target image

In [ ]:
# get image path
img_path = 'cat.jpg'
target_image = image_loader(img_path, (512, 512))
imshow(target_image)

In [ ]:
# Get same size noise image
noise = torch.empty_like(target_image).uniform_(0, 1).to(device)
imshow(noise)

In [ ]:
loss = nn.MSELoss() # Set Loss function
iters = 100 # Set iteration numbers
lr = 1e4

print('[ Start ]')
imshow(noise)

for i in range(iters):
    # set required_grad True to trace torch.Tensor
    noise.requires_grad = True
    
    # Get gradient
    output = loss(noise, target_image)
    output.backward()
    
    # Update to make loss smaller
    gradient = lr * noise.grad
    
    # clip noise_image [0, 1]
    noise = torch.clamp(noise - gradient, min=0, max=1).detach_() # detach -> stop Tracking
    
    if (i + 1) % 10 == 0:
        print(f'[ Step: {i+1} ]')
        print(f'Loss: {output}')
        imshow(noise)
    

# Get Image for Style transfer

In [ ]:
# Get content image, style image
content_img = image_loader('content_img_3.jpg', (512, 640))
style_img = image_loader('style_img_1.jpg', (512, 640))

imshow(content_img)
imshow(style_img)

# Load CNN

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()
print(cnn)

In [ ]:
# Normalization
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = mean.clone().view(-1,1,1)
        self.std = std.clone().view(-1,1,1)
        
    def forward(self, img):
        return (img - self.mean) / self.std

# Style Reconstruction
* Make noise image to be similar with content and style image
* Get 5 style_layers and 1 content_layers

In [ ]:
def gram_matrix(input):
    # a: batch_size, b: feature_map_numbers, (c,d) -> feature map dimension
    a, b, c, d = input.size()
    
    # i == feature_map_numbers, j == position
    features = input.view(a * b, c * d)
    
    G = torch.mm(features, features.t())
    return G.div(a*b*c*d)

# Style loss
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
        
    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [ ]:
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# 스타일 손실(style loss)을 계산하는 함수
def get_style_losses(cnn, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 style loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, style_losses


In [ ]:
def style_reconstruction(cnn, style_img, input_img, iters):
    model, style_losses = get_style_losses(cnn, style_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()], lr=0.1)
    
    print('[Start]')
    imshow(input_img)
    
    run = [0]
    while run[0] <= iters:
        
        def closure():
            input_img.data.clamp_(0, 1)
            
            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            
            for sl in style_losses:
                style_score += sl.loss
            
            style_score *= 1e6
            style_score.backward()
            
            run[0] += 1
            if run[0] % 50 == 0:
                print(f'[step : {run[0]} / Style loss : {style_score.item()}]')
                imshow(input_img)
            return style_score
        
        optimizer.step(closure)
    
    # clip image [0, 1]
    input_img.data.clamp_(0, 1)
    
    return input_img

In [ ]:
input_img = torch.empty_like(content_img).uniform_(0,1).to(device)
imshow(input_img)

In [ ]:
# Run Style Reconstruction
output = style_reconstruction(cnn, style_img=style_img, input_img=input_img, iters=3000)

# Content Reconstruction

In [ ]:
# Class for Content Loss
class ContentLoss(nn.Module):
    def __init__(self, target, ):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
        
    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
content_layers = ['conv_4']

# 콘텐츠 손실(content loss)을 계산하는 함수
def get_content_losses(cnn, content_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

    # 마지막 content loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses

In [ ]:
def content_reconstruction(cnn, content_img, input_img, iters):
    model, content_losses = get_content_losses(cnn, content_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()], lr=0.1)

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0

            for cl in content_losses:
                content_score += cl.loss

            content_score.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()}]")
                imshow(input_img)
            
            return content_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img



In [ ]:
input_img = torch.empty_like(content_img).uniform_(0,1).to(device)
imshow(input_img)

In [ ]:
# Run Content Reconstruction
output = content_reconstruction(cnn, content_img=content_img, input_img=input_img, iters=3000)

# Style Transfer
* use Content Loss and Style Loss

In [ ]:
content_layers = ['conv_4']
style_layers = ['conv_1', 'conv_3', 'conv_5', 'conv_7', 'conv_9']

# Style Transfer 손실(loss)을 계산하는 함수
def get_losses(cnn, content_img, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses, style_losses

In [ ]:
def style_transfer(cnn, content_img, style_img, input_img, iters, lr, patience):
    model, content_losses, style_losses = get_losses(cnn, content_img, style_img, input_img)
    
    optimizer = optim.LBFGS([input_img.requires_grad_()], lr=lr)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience)
    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0
            style_score = 0

            for cl in content_losses:
                content_score += cl.loss
            for sl in style_losses:
                style_score += sl.loss

            style_score *= 1e5
            loss = content_score + style_score
            loss.backward()

            run[0] += 1
            if run[0] % 100 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()} / Style loss: {style_score.item()}]")
                imshow(input_img)
                save_image(input_img.cpu().detach()[0], f'{run[0]}.png') 
            
            return content_score + style_score
        
        optimizer.step(closure)
        scheduler.step(closure())

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
# Get content image, style image
content_img = image_loader('content_img_3.jpg', (512, 640))
style_img = image_loader('style_img_1.jpg', (512, 640))
input_img = torch.empty_like(content_img).uniform_(0,1).to(device)

imshow(content_img)
imshow(style_img)
imshow(input_img)

# Run style-transfer

In [ ]:
output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, iters=6000, lr=1, patience=15)

# Another samples

In [ ]:
# Get content image, style image
content_img = image_loader('content_img_9.jpg', (650, 1000))
style_img = image_loader('style_img_7.png', (650, 1000))
input_img = torch.empty_like(content_img).uniform_(0,1).to(device)

imshow(content_img)
imshow(style_img)
imshow(input_img)

output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, iters=6000, lr=1, patience=15)

In [ ]:
content_img = image_loader('content_img_8.jpg', (650, 1000))
style_img = image_loader('style_img_5.jpg', (650, 1000))
input_img = torch.empty_like(content_img).uniform_(0,1).to(device)

imshow(content_img)
imshow(style_img)
imshow(input_img)

output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, lr=1, iters=10000, patience=15)